<a href="https://colab.research.google.com/github/wizzhi/publicDrafts/blob/main/PO_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# using BERT calssification
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 16.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.6 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification
# AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import numpy as np
import pandas as pd

from sklearn import preprocessing 
labelEncoder = preprocessing.LabelEncoder() 

import io

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print( f"using {device} !!!" )

using cuda !!!


In [3]:
# upload data files
from google.colab import files
uploaded = files.upload()


Saving TRAIN200K_matched_short_L4.csv to TRAIN200K_matched_short_L4.csv
Saving unspscL4.csv to unspscL4.csv


In [4]:

# data perperation
unspsc_level = "L4"
file_from = "TRAIN200K_matched_short_" + unspsc_level

df = pd.read_csv( io.StringIO( uploaded['unspsc'+unspsc_level+'.csv'].decode('utf-8')))
print( f"Master data loaded. {df.shape}" )
df.CODE = labelEncoder.fit_transform( df.CODE )
unspsc_CODE_list = df.CODE.tolist()
unspsc_NAME_list = df.NAME.tolist()

df = pd.read_csv( io.StringIO( uploaded[file_from+'.csv'].decode('utf-8')))
print( f"Customer data loaded. {df.shape}" )
# df = df.sample(60000)
print( f"cut to {df.shape}" )

df.CODE = labelEncoder.transform(df.CODE)


# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

documents = df.NAME.values
labels = df.CODE.values


from sklearn.model_selection import train_test_split

train_texts, valid_texts, train_labels, valid_labels = train_test_split(documents, labels, test_size=0.2, shuffle=True)
print( f"Split into training set {train_texts.shape} & test set {valid_texts.shape}" )


# adding standard UNSPSC to traning set
train_texts = np.append(train_texts, unspsc_NAME_list)
train_labels = np.append(train_labels, unspsc_CODE_list)
print( f"Masted data added into training set {train_labels.shape}" )


class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=64)
valid_encodings = tokenizer(valid_texts.tolist(), truncation=True, padding=True, max_length=64)

train_dataset = BertDataset(train_encodings, train_labels)
valid_dataset = BertDataset(valid_encodings, valid_labels)


Master data loaded. (46137, 2)
Customer data loaded. (126942, 3)
cut to (126942, 3)


Split into training set (101553,) & test set (25389,)
Masted data added into training set (147690,)


In [6]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=46137)

from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation (20)
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=3000,               # log & save weights each logging_steps
    save_steps=12000,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 147690
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 55386


Step,Training Loss,Validation Loss,Accuracy
3000,7.500500,4.893307,0.370830
6000,6.349300,4.109382,0.455197
9000,6.008500,3.610897,0.511875
12000,5.747200,3.266810,0.551577
15000,5.628900,3.024083,0.585135
18000,5.469000,2.876664,0.602860


***** Running Evaluation *****
  Num examples = 25389
  Batch size = 10
***** Running Evaluation *****
  Num examples = 25389
  Batch size = 10
***** Running Evaluation *****
  Num examples = 25389
  Batch size = 10
***** Running Evaluation *****
  Num examples = 25389
  Batch size = 10
Saving model checkpoint to ./results/checkpoint-12000
Configuration saved in ./results/checkpoint-12000/config.json
Model weights saved in ./results/checkpoint-12000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 25389
  Batch size = 10
***** Running Evaluation *****
  Num examples = 25389
  Batch size = 10


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 16000
  Batch size = 20


{'epoch': 3.0,
 'eval_accuracy': 0.672625,
 'eval_loss': 2.30497670173645,
 'eval_runtime': 76.9429,
 'eval_samples_per_second': 207.946,
 'eval_steps_per_second': 10.397}